In [623]:
import es_core_news_sm
from gensim.models import Word2Vec, KeyedVectors
import numpy as np
import regex
import pandas as pd
from scipy.spatial.distance import cosine
import os
import nltk

In [624]:
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/parker/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /home/parker/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /home/parker/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /home/parker/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /home/parker/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /home/parker/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to

True

In [625]:
# Funcion para entrenar el modelo
def EntrenarModelo(oraciones,NombreModelo):
    model = Word2Vec(oraciones,vector_size=1250, window=20, min_count=1)
    model.save(NombreModelo)
# Funcion para cargar un modelo ya existente
def CargarModelo(NombreModelo):
   modelo = Word2Vec.load(NombreModelo)
   vocabulario = [term for term in modelo.wv.key_to_index]  
   return(modelo,vocabulario)

In [626]:
# Obtener el emebdding de un TEXTO
def ObtenerEmbeddingTexto(modelo, texto):
   Lista_vectores = []
   for w in nltk.sent_tokenize(texto, language="spanish"):
       # Verificar que la oracion w exista en el modelo
       try:
           modelo.wv[w]
       except KeyError:
           continue
       # Obtener vector de la oracion
       vec = modelo.wv[w]
       Lista_vectores.append(vec)
   embedding_oraciones = np.array(Lista_vectores)
   if (len(embedding_oraciones) > 0):
        embedding_texto = embedding_oraciones.mean(axis=0)
   else:
        embedding_texto = np.zeros(modelo.vector_size)
   return(embedding_texto) 

In [627]:
# Funcion para crear el corpus
def CrearCorpus(path):
  directorio = os.listdir(path)
  corpus = []
  doc_id = []  
  for filename  in directorio:
     texto = open(path+filename,'r',encoding="UTF-8").read()
     corpus.append(texto)
     doc_id.append(filename)
  return(corpus,doc_id)
# Preprocesamiento de textos
def PreProcesarTextos(textos):
    texto_limpio = []
    # Para cada texto tokenizar por oraciones
    for texto in textos:
        if len(texto) != 0:
            texto_limpio.append(nltk.sent_tokenize(texto, language="spanish"))
    return texto_limpio

In [628]:
def CrearDiccionario(lista,claves):
   dicc = {}
   for  v in range(0,len(claves)):
      dicc[claves[v]] = lista[v]
   return(dicc)

In [629]:
PATH = "DiscursosOriginales/"
# Creamos el corpus a partir de todos los textos en el directiorio PATH
corpus,docID = CrearCorpus(PATH)
# Se divide cada texto en oraciones
oraciones = PreProcesarTextos(corpus)
CorpusConClave  = CrearDiccionario(corpus,docID)


In [630]:
# Entrenamos el modelo con los textos tokenizados por oracion y lo llamamos mi_word2vec
EntrenarModelo(oraciones,'mi_word2vec')

In [631]:
# cargamos el modelo y el vocabulario
modelo, vocabulario = CargarModelo('mi_word2vec')

In [633]:
# Ordenar oraciones por puntaje
def centroide(modelo, oraciones):
    embedding = []
    for oracion in oraciones:
        embedding.append(modelo.wv[oracion])
    return np.mean(embedding, axis=0)
def OrdenarOraciones(oraciones):
    puntaje = []
    centroidee = centroide(modelo, oraciones)
    oraciones.sort(key=lambda oracion: cosine(modelo.wv[oracion],centroidee), reverse=True)
    return oraciones

In [634]:
# Metodo summarization imlementado en python
def generar_resumen(O,N,P,U):
    O = OrdenarOraciones(O)
    largo = 1
    Resumen = [""]*(N+1)
    M = len(O)
    for i in range(M):
        if (largo > N): return Resumen[1:N+1]
        Vo = modelo.wv[O[i]]
        incluirOracion = False
        for j in range(largo):
            try:
                Vr = modelo.wv[Resumen[j]]
            except KeyError:
                Vr = np.zeros(len(Vo))
            Sim = 1-cosine(Vo, Vr)
            if Sim > U and (O[i] not in Resumen):
                incluirOracion = True
        if incluirOracion == True:
            Resumen[largo] = O[i]
            largo+=1
    return Resumen[1:N+1]

In [635]:
# Generamos un resumen de un texto aleatorio
text = open('./DiscursosOriginales/72186.txt', encoding='UTF-8').read()
oraciones = PreProcesarTextos([text])[0]
resumen = generar_resumen(oraciones, 10, 0,0.2)
resumen

/home/parker/Documents/env/jupy/lib64/python3.10/site-packages/scipy/spatial/distance.py:620: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


['Esa entrega que han demostrado a lo largo de la historia de la Fuerza Aérea los hombres y mujeres que componen esta institución, y así lo han demostrado cada vez que la patria los ha necesitado.',
 'Las trágicas muertes de los tenientes Mery, Bello, Vidal, Marsh, el sargento Menadier, el guardiamarina Zañartu, el subteniente Wall y de tantos, fueron las semillas que lograron echar raíces profundas y que han germinado dando grandes frutos en la historia y también en el futuro de nuestra Fuerza Aérea.',
 'Basta con mencionar, y ya lo dijo el Comandante en Jefe, la ayuda que prestaron con ocasión de tantos desastres naturales a lo largo de nuestra historia, los puentes aéreos para llevar alimentos, medicamentos y servicios básicos a miles de compatriotas que, de otra forma, tal vez no habrían logrado sobrevivir.',
 'Y es ese mismo compromiso y entrega que hoy hemos reconocido al entregar la condecoración “General Diego Aracena Aguilar” a seis destacados oficiales y suboficiales y civile

['Muchas gracias.',
 '¡Viva Chile!',
 '¿Qué significa eso?',
 '¿Por qué?',
 'No.',
 '¿Qué significa esto?',
 'Muy buenos días.',
 'Muy buenas noches y muchas gracias.',
 'Muy buenas tardes.',
 '2.',
 '3.',
 'No hay mejor policía que el alumbrado público; no hay mejor desinfectante que la luz solar.',
 'Muchas gracias y muy buenas noches.',
 '3º.',
 '2º.',
 'Y podríamos seguir.',
 '¿Qué estamos haciendo?',
 'Por supuesto que no.',
 'Gracias.',
 '¿Qué es el Hospital Digital?',
 '4.',
 '1.',
 '(risas).',
 '¡Muchas gracias!',
 'Muchas gracias',
 'Todo lo contrario.',
 '4º.',
 'Pero eso no es todo.',
 '¿En qué consiste este proyecto?',
 'Muy buenas tardes y muchas gracias.',
 'Buenas noches y muchas gracias.',
 'Eso no es verdad.',
 '¿Qué significa ser patriota?',
 '¿En qué consiste?',
 'Hicimos un gran esfuerzo -y esto fue un esfuerzo consensuado- por identificar las principales brechas y carencias, porque junto con poner en marcha los motores, tenemos que remover los cuellos de botella, l